# Road Follower - Train Model

In this notebook we will train a neural network to take an input image, and output a set of x, y values corresponding to a target.

We will be using PyTorch deep learning framework to train ResNet18 neural network architecture model for road follower application.

### Download and extract data

Before you start, you should upload the ``road_following_<Date&Time>.zip`` file that you created in the ``data_collection.ipynb`` notebook on the robot. 

> If you're training on the JetBot you collected data on, you can skip this!

You should then extract this dataset by calling the command below:

## 若需要解壓縮訓練資料集的資料夾，再進行以下程式功能。
### 執行前，請先取消註解符號 # ，再進行執行。
### 解壓縮需要時間，請耐心等候。

In [ ]:
# !unzip -q road_following.zip

# 請設定訓練用資料集的資料夾名稱
## 請務必記得名稱文字前後要加引號 --> ' 

In [1]:
dataset_path = 'seangao_0606'

# 請設定訓練完成後的模型檔(pth檔)名稱
## 請注意!
### 設定檔案名稱要記得指定副檔名為 --> .pth
### 設定檔案名稱要記得在名稱字串前後均加上引號 --> '

In [2]:
model_file = 'seangao_0525_v1_606.pth'

# 設定神經網路模型訓練的回合數
## 建議至少20回合以上

In [3]:
set_epochs = 20

# <font color="#dd0000">請注意!
# <font color="#dd0000">以下所有程式區塊內容，若非專業人士請勿任意修改程式碼。
# <font color="#dd0000">直接執行所有區塊即可。

In [4]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
import glob
import PIL.Image
import os
import numpy as np

## 設定功率模式到5W

In [5]:
import os
password = 'jetbot'
command_set_power = "sudo -S nvpmodel -m1"
print(command_set_power)
os.system('echo %s | %s'%(password,command_set_power))

sudo -S nvpmodel -m1


0

## 開啟散熱風扇

In [6]:
import os
password = 'jetbot'
command = "sudo -S sh -c 'echo 255 > /sys/devices/pwm-fan/target_pwm'"
print(command)
os.system('echo %s | %s'%(password,command))

sudo -S sh -c 'echo 255 > /sys/devices/pwm-fan/target_pwm'


0

You should see a folder named ``dataset_all`` appear in the file browser.

### Create Dataset Instance

Here we create a custom ``torch.utils.data.Dataset`` implementation, which implements the ``__len__`` and ``__getitem__`` functions.  This class
is responsible for loading images and parsing the x, y values from the image filenames.  Because we implement the ``torch.utils.data.Dataset`` class,
we can use all of the torch data utilities :)

We hard coded some transformations (like color jitter) into our dataset.  We made random horizontal flips optional (in case you want to follow a non-symmetric path, like a road
where we need to 'stay right').  If it doesn't matter whether your robot follows some convention, you could enable flips to augment the dataset.

In [7]:
def get_x(path):
    """Gets the x value from the image filename"""
    return (float(int(path[3:6])) - 50.0) / 50.0

def get_y(path):
    """Gets the y value from the image filename"""
    return (float(int(path[7:10])) - 50.0) / 50.0

class XYDataset(torch.utils.data.Dataset):
    
    def __init__(self, directory, random_hflips=False):
        self.directory = directory
        self.random_hflips = random_hflips
        self.image_paths = glob.glob(os.path.join(self.directory, '*.jpg'))
        self.color_jitter = transforms.ColorJitter(0.3, 0.3, 0.3, 0.3)
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        
        image = PIL.Image.open(image_path)
        x = float(get_x(os.path.basename(image_path)))
        y = float(get_y(os.path.basename(image_path)))
        
        if float(np.random.rand(1)) > 0.5:
            image = transforms.functional.hflip(image)
            x = -x
        
        image = self.color_jitter(image)
        image = transforms.functional.resize(image, (224, 224))
        image = transforms.functional.to_tensor(image)
        image = image.numpy()[::-1].copy()
        image = torch.from_numpy(image)
        image = transforms.functional.normalize(image, [0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        
        return image, torch.tensor([x, y]).float()
    
#dataset = XYDataset('dataset_1031_100', random_hflips=False)
dataset = XYDataset(dataset_path, random_hflips=False)

### Split dataset into train and test sets
Once we read dataset, we will split data set in train and test sets. In this example we split train and test a 90%-10%. The test set will be used to verify the accuracy of the model we train.

In [8]:
test_percent = 0.1
num_test = int(test_percent * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - num_test, num_test])

### Create data loaders to load data in batches

We use ``DataLoader`` class to load data in batches, shuffle data and allow using multi-subprocesses. In this example we use batch size of 64. Batch size will be based on memory available with your GPU and it can impact accuracy of the model.

In [9]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

### Define Neural Network Model 

We use ResNet-18 model available on PyTorch TorchVision. 

In a process called transfer learning, we can repurpose a pre-trained model (trained on millions of images) for a new task that has possibly much less data available.


More details on ResNet-18 : https://github.com/pytorch/vision/blob/master/torchvision/models/resnet.py

More Details on Transfer Learning: https://www.youtube.com/watch?v=yofjFQddwHE 

In [10]:
model = models.resnet18(pretrained=True)

ResNet model has fully connect (fc) final layer with 512 as ``in_features`` and we will be training for regression thus ``out_features`` as 1

Finally, we transfer our model for execution on the GPU

In [11]:
model.fc = torch.nn.Linear(512, 2)
device = torch.device('cuda')
model = model.to(device)

### Train Regression:

We train for 20 epochs and save best model if the loss is reduced. 

In [ ]:
NUM_EPOCHS = set_epochs
BEST_MODEL_PATH = model_file
best_loss = 1e9

optimizer = optim.Adam(model.parameters())

for epoch in range(NUM_EPOCHS):
    
    model.train()
    train_loss = 0.0
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        train_loss += loss
        loss.backward()
        optimizer.step()
    train_loss /= len(train_loader)
    
    model.eval()
    test_loss = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = F.mse_loss(outputs, labels)
        test_loss += loss
    test_loss /= len(test_loader)
    
    print('round %d: %f, %f' % (epoch, train_loss, test_loss))
    if test_loss < best_loss:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_loss = test_loss
print('Done!')

round 0: 0.853579, 6.458896
round 1: 0.392428, 35.598942
round 2: 0.119068, 1.204937
round 3: 0.091483, 0.419807
round 4: 0.077310, 0.047666
round 5: 0.063682, 0.068176
round 6: 0.061449, 0.024183
round 7: 0.057984, 0.062527
round 8: 0.034575, 0.022811
round 9: 0.045214, 0.055236
round 10: 0.034726, 0.037151
round 11: 0.042908, 0.026497
round 12: 0.034597, 0.027309
round 13: 0.053331, 0.028218
round 14: 0.060618, 0.025248
round 15: 0.025403, 0.031067
round 16: 0.031484, 0.024586
round 17: 0.019004, 0.019550


Once the model is trained, it will generate ``best_steering_model_xy.pth`` file which you can use for inferencing in the live demo notebook.

If you trained on a different machine other than JetBot, you'll need to upload this to the JetBot to the ``road_following`` example folder.

## 關閉風扇

In [13]:
import os
password = 'jetbot'
command = "sudo -S sh -c 'echo 0 > /sys/devices/pwm-fan/target_pwm'"

print(command)

os.system('echo %s | %s'%(password,command))

sudo -S sh -c 'echo 0 > /sys/devices/pwm-fan/target_pwm'


0